In [12]:
from pyspark import sql
from lib import common_functions
from lib import configuration

In [ ]:
spark = common_functions.get_spark_session('football')
spark.active()

Variables

In [ ]:
goals_path = configuration.football_input_path+'CRN Temp - Goals.csv'
win_path = configuration.football_input_path+'win_btts.csv'

goals_path

'/home/jovyan/code/files/input/CRN Temp - Goals.csv'

Dataframes

In [39]:
df_goals = spark.read.format("csv").options(header='true', inferSchema='true', delimiter=',').load(goals_path)
df_goals = df_goals.drop('7.47%').drop('47.04%')
df_win = spark.read.format("csv").options(header='true', inferSchema='true', delimiter=',').load(win_path)

Fix Column names

In [41]:
col_dict_to_update = {'Fixture (O 1.5, if not stated)':'Fixture','Over 1.5 Goals':'Over One','Over 2.5 Goals':'Over Two','BTTS %':'BTTS Perc','Over 2.5 Goals %':'Over Two Perc','WIN %':'WIN Perc'}

for x, y in col_dict_to_update.items():
    df_goals = df_goals.withColumnRenamed(x,y)
    
df_goals = df_goals.select([sql.functions.col(x).alias(x.replace(' ', '_')) for x in df_goals.columns])

df_goals.show()

+---+--------------------+--------+------------+------------+--------+-----+----------+----------+----+--------+--------+------+--------+--------+---------+-------------+--------+----------+----+--------------+
| ID|             Fixture|Home_For|Home_Against|Away_Against|Away_For|Total|Home_Goals|Away_Goals|BTTS|Over_One|Over_Two|Result|Home_Win|Away_Win|BTTS_Perc|Over_Two_Perc|WIN_Perc|windrawwin|Odds|windrawwinPred|
+---+--------------------+--------+------------+------------+--------+-----+----------+----------+----+--------+--------+------+--------+--------+---------+-------------+--------+----------+----+--------------+
|  1|Borussia M'gladba...|      15|           6|          11|       5|   37|         1|         0|   0|       0|       0|     1|       1|       0|     NULL|         NULL|    NULL|      NULL|NULL|          NULL|
|  2|      Lyon vs Troyes|      14|           8|          13|       6|   41|         4|         1|   1|       1|       1|     1|       1|       0|     NULL|

Filters

In [53]:
df_history = df_goals.filter(df_goals.Home_Goals.isNotNull()).filter(df_goals.BTTS_Perc.isNotNull()).filter(df_goals.Over_Two_Perc.isNotNull()).filter(df_goals.windrawwin.isNotNull())
df_predict = df_goals.filter(df_goals.Home_Goals.isNull())
df_history.show()

df_history.count()

df_history.toPandas().to_csv('files/output/export.csv')

+-----+--------------------+--------+------------+------------+--------+-----+----------+----------+----+--------+--------+------+--------+--------+---------+-------------+--------+----------+----+--------------+
|   ID|             Fixture|Home_For|Home_Against|Away_Against|Away_For|Total|Home_Goals|Away_Goals|BTTS|Over_One|Over_Two|Result|Home_Win|Away_Win|BTTS_Perc|Over_Two_Perc|WIN_Perc|windrawwin|Odds|windrawwinPred|
+-----+--------------------+--------+------------+------------+--------+-----+----------+----------+----+--------+--------+------+--------+--------+---------+-------------+--------+----------+----+--------------+
|  995|Sparta Praha vs P...|      17|           6|           7|       5|   35|         2|         1|   1|       1|       1|     1|       1|       0|     0.54|         0.61|     0.7|       2-0|NULL|             1|
|  996|Manchester City v...|      15|           3|           6|      12|   36|         4|         1|   1|       1|       1|     1|       1|       0|